# 2.10: Inspecting the `exp` directories

`run_train_phones.sh` 파일을 실행하면 `exp` 디렉토리 아래에 각각의 layer에서 생성된 acoustic model을 저장하게 됩니다. 어떠한 파일들이 있는지 살펴보겠습니다. 

In [ ]:
ls exp

## `exp/monophones`

이 디렉토리에는 *첫 번째* layer인 `monophones`에서 생성된 파일들이 들어있습니다. 

In [ ]:
ls exp/monophones

### `exp/monophones/log`

`monophones`에 해당하는 acoustic model을 생성하는 과정에 대한 *log* 파일들이 이 디렉토리에 저장되어 있습니다. 첫 번째 번호는 몇 번째 **iteration**인지를 나타내는 것이고, 마지막 번호는 어떠한 **thread**에서 처리가 되었는지를 나타내는 것입니다. 특정 로그 파일이 다른 로그 파일들에 비해 중요한 정보를 담고 있을 수 있으며, 에러가 발생하였을 경우에 **해당 로그 파일**은 중요한 내용을 담고 있을 수 있습니다. 

In [ ]:
ls exp/monophones/log

### `num_jobs`

`kaldi` 디렉토리에 있는 `num_jobs` 파일입니다. 단순한 `숫자`를 담고 있으며, 병렬처리에 몇 개의 thread가 사용되었는지를 나타냅니다. 

In [ ]:
cat exp/monophones/num_jobs

### `cmvn_opts`

`_opts`로 끝나는 파일들은 스크립트에서 사용하는 *hyperparameters*에 관련된 정보를 담고 있는 `options` 파일입니다. `kaldi_config.json` 파일에서 `non_vanilla_*` 옵션으로 추가할 때 사용하는 것과 같은 형식으로 나타납니다. 



```
--variable_name [variable_value]
```

이 pipeline에서 `cmvn_opts`는 비어있습니다. 

In [ ]:
cat exp/monophones/cmvn_opts

### `{40,final}.occs`

이 파일은 "per-transition-id occupation counts" 정보를 담고 있으며, `kaldi`를 작성한 사람의 포스트를 참조하면 이 파일은 거의 사용되지 않는다고 합니다. 그러므로 건너뛰도록 하겠습니다. 

파일 구조만 살펴보면, `40.occs`와 `final.occs` 파일을 볼 수 있습니다. `40`이라는 숫자가 의미하는 것은, 해당 정보가 각 `iteration`마다 업데이트 된다는 것입니다. 하지만 맨 마지막 `iteration`을 제외한 나머지 파일들은 삭제된 것을 알 수 있습니다. 그 이후, `final.occs` 파일은 `40.occs` 파일의 바로가기 파일입니다. 

**Note:** `kaldi`에서는 이와 같은 바로가기(`symbolic link`)를 자주 사용합니다. 아래에 나오는 `.mld` 파일 또한 바로가기를 사용하였습니다. 

In [ ]:
ls -lah exp/monophones | grep occs

### `{40,final}.mdl`

`.mdl` 파일은 실제로 생성된 acoustic model 파일입니다. 이 `.mdl` 파일을 decoding 과정에서도 사용할 수 있습니다. 각각의 "layer"는 각 layer에 해당하는 `.mdl` 파일을 생성합니다. 

`.mdl` 파일의 자세한 내용은 다른 `notebook`에서 다루도록 하겠습니다. `.mdl` 파일 내부는 `show-transitions` 기능(`path.sh` 파일을 "source"로 추가하여야 합니다)을 사용하여서 "human-readable" 포맷으로 변환할 수 있습니다. 


In [ ]:
. ${KALDI_INSTRUCTIONAL_PATH}/path.sh
show-transitions

In [ ]:
show-transitions \
    data/lang/phones.txt \
    exp/monophones/final.mdl \
    | head

### `fsts.*.gz`

이 파일은 training 데이터에서 만들어진 `FST` 정보를 담고 있습니다. 병렬처리에서 각각의 thread는 하나의 tsts.\*.gz 파일을 생성합니다. 나중에 `FST` 파일은 더욱 자세하게 다룰 예정이므로, 지금은 넘어가도록 하겠습니다. 

### `ali.*.gz`

이 파일은 각각의 프레임을 해당하는 phone과 연결하는 정보를 담고 있습니다. 이와 비슷한 `ali.*.gz` 파일을 `MFCC`를 확인할 때 사용했던 것을 확인하시면 좋을 것 같습니다. `ali-to-phones` 기능을 사용하여서 파일 안의 내용을 phone 리스트로 변환하여 출력할 수 있습니다. 자세한 내용은 나중에 다시 살펴보도록 하겠습니다. 

In [ ]:
ali-to-phones

**Note:** 대상 파일들은 압축된 파일들입니다(`gzip` 파일). 그렇기 때문에 실제 `binary` 파일에 접근하기 위해서는 압축을 풀어줄 필요가 있습니다. 아래의 명령어를 통해서 어떻게 `gzip` 파일의 압축을 풀 수 있는지 볼 수 있습니다. 

**Note:** 중간 과정에서 `int2sym.pl` 스크립트를 사용한 것을 알 수 있습니다. 그 이유는 `fsts-to-transcripts` 파일의 출력은 인덱스의 형태이기 때문입니다. `int2sym.pl` 스크립트를 이용해서 실제 단어로 변환해볼 수 있습니다. 

In [ ]:
ali-to-phones \
    --per-frame=true \
    exp/monophones/final.mdl \
    "ark:gzip -cd exp/monophones/ali.1.gz|" \
    "ark,t:|int2sym.pl -f 2- data/lang/phones.txt" \
    | head -n1

### `tree`

이 파일은 phone을 그룹화 할 때 사용하는 *decision tree*의 정보를 담고 있습니다. 자세한 내용은 나중에 다시 살펴볼 것입니다. *Decision tree*의 형태는 `draw-tree` 기능을 사용하여서 시각화 할 수 있습니다. 

In [ ]:
draw-tree

아래 명령어를 실행하면 `decision tree`를 `.jpg` 포맷으로 `exp/monophones/tree`에 저장하게 됩니다. 그리고 그 다음 셀에서는 `.jpg` 파일을 랜더링 하여 출력하게 됩니다. 

In [ ]:
draw-tree \
    data/lang/phones.txt \
    exp/monophones/tree \
    | dot -Tpng -Gsize=8,10.5 > exp/monophones/tree.jpg

![tree](exp/monophones/tree.jpg)

안의 내용을 하나하나 살펴보는 것은 쉽지 않습니다. 하지만 전체적으로 어떠한 구조를 가지고 있는지는 살펴볼 수 있습니다. 

## `exp/monophones_aligned`


이 폴더에는 `monophones` layer에서 생성된 파일들이 저장되어 있습니다. 이전에 `monophones`에서 살펴본 파일들과 동일합니다. 

In [ ]:
ls exp/monophones_aligned

## `exp/triphones`

이 폴더에는 *두 번째* layer인 `triphones` layer에서 생성된 파일들이 저장되어 있습니다. `monophones`와 비교하면 두 개의 파일이 추가로 생성되어 있습니다. 두 파일은 `questions.{int, qst}` 입니다. 


In [ ]:
ls -lah exp/triphones

### `questions.{int,qst}`

이 파일은 `decision tree`에 쿼리 된 "questions"을 담고 있습니다. 그리고 "questions"라는 것은 그룹화 되어있는 phone 들을 의미합니다. `.int` 파일에서 각각의 줄은 하나의 그룹을 나타냅니다. (`.qst` 파일은 같은 내용을 표현하는 `kaldi` 바이너리 파일입니다.)

Decision tree의 *성능*은 나중에 살펴보도록 하겠습니다. 

**Note:** 이 파일은 `int2sym.pl` 파일을 이용하여서 살펴보아야 합니다. 그 이유는 `.int` 파일은 실제 `phone`이 아니라 해당 `phone`의 `index`를 담고 있기 때문입니다. 

In [ ]:
cat exp/triphones/questions.int | int2sym.pl -f 1- data/lang/phones.txt | head -n3

## `exp/triphones_aligned`

`monophones_aligned`와 마찬가지로 이 디렉토리에는 `triphones` layer를 생성하는 과정에서 만들어진 파일들이 저장되어 있습니다. `triphones`에서 설명한 내용과 동일한 파일들이 있습니다. 

In [ ]:
ls exp/triphones_aligned

## `exp/triphones_lda`

이 디렉토리에는 *세 번째* layer인 `LDA_MLLT` 과정에서 생성된 파일들이 저장되어 있습니다. 

In [ ]:
ls -lah exp/triphones_lda

### `*.mat`

아직까지 내용을 확인하지 않은 파일은 `.mat` 파일입니다. `mat`은 `matrix`의 약자입니다. 이 파일은 가지고 있는 데이터셋에 `LDA`를 적용하기 위한 `matrix`가 저장되어 있습니다. 이전에 살펴본 것처럼 `final.mat` 파일은 마지막에 생성된 `.mat` 파일의 `symbolic link`입니다. 이 경우, `12.mat` 파일에 바로가기가 연결되어 있습니다. 

`copy-matrix` 파일에 `--binary=false` 옵션을 주어서 `kaldi` 바이너리 파일을 텍스트 형태로 출력할 수 있습니다. 

In [ ]:
copy-matrix

In [ ]:
copy-matrix \
    --binary=false \
    exp/triphones_lda/final.mat \
    "| head -n3"

어떠한 내용인지 확인하기는 어렵지만, matrix의 구조가 우리가 생각하는 것과 같은지 `matrix-dim` 기능을 이용하여서 확인할 수 있습니다. 

In [ ]:
matrix-dim

In [ ]:
matrix-dim exp/triphones_lda/final.mat

## `exp/triphones_lda_aligned`

다른 `*_aligned` 디렉토리와 마찬가지로, 이 디렉토리에는 `LDA` 과정에서 생성된 파일들이 저장되어 있습니다. `trans.*` 라는 파일들을 제외한 나머지 파일들은 `triphones_lda` 디렉토리에 있는 파일들입니다. 

In [ ]:
ls exp/triphones_lda_aligned